## Goal
I am gonna watch videos on algo trading, so that I may integrate it into the actual algobot and make a comprehensive prediction and shit 

pip freeze > requirements.txt

### Getting and parsing data to work with

In [1]:
# Complete package installation for Jupyter + Plotly
%pip install yfinance plotly pandas numpy talib-binary ipywidgets

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from trading_functions import create_individual_portfolio_charts, rsi_calculator

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
#Define the portfolio 
portfolio = {
    'czoof': 0.2,
    'FCNVX': 0.995,
    'fdig': 0.925,
    'fdkvx': 0.862,
    'feddx': 0.961,
    'ffrhx': 3.247,
    'fifnx': 0.969,
    'fitlx': 1.151,
    'flowx': 0.838,
    'fsenx': 0.326,
    'LION': 2,
    'OLLI': 0.175,
    'UNP': 0.1,
    'VZ': 0.5,
    'WPP': 0.3
}

In [5]:
# yfinance style (for stocks)
ticker = list(portfolio.keys())  
period = "1000d"         # Instead of 'num_bars' (or use start/end dates)
interval = "1d"          # Instead of 'timeframe'
# No 'start_pos' - uses dates instead
start_date = None  # Use None for the most recent data
end_date = None    # Use None for the most recent data

### RSI
An rsi is trying to buy when prices are over sold and sell when prices are overbought. It is a mean reversion strategy. when rsi drops below over

https://youtu.be/I5unWZBldus?si=15gKhJBv264YiDUd

https://youtu.be/vT0-eLOw5Uk

so that you can see when the strategy is doing well and when the strategy is doing poorly(You can probably take this data and then plug it into a bayesian thing, once you finish chapter 4)

You can probably use some type of bayesian expression to adjust the bounds of the rsi

You can possibly do something with a fourier transform with this, and maybe integrate the peaks and shit like that of the actual stocks here.....

Figure out where the less than 30 and the 70 are coming from and even the 14 days. See if there is a way to get a bayesian statement or something to automatically adjust it depending on the current or maybe even past data, same with the atr part. You might need to finish chapter 4 or 5 for this, so maybe later

maybe see if there is a way to combine mcmc with the iterations of historical data so that we can do multiple iterations than would be humanly possible in a small amount of time. 

#### Visualize the rsi indicator on the entire portfolio of stocks

In [6]:
for symbol in ticker:
    data = yf.download(symbol, period=period, interval=interval, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    if 'Date' not in data.columns:
        data.rename(columns={data.columns[0]: 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    data = rsi_calculator(data)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data['Date'].dt.strftime('%Y-%m-%d'),
        y=data['rsi_14'],
        mode='lines',
        name='RSI (14)',
        line=dict(color='orange', width=2)
    ))
    fig.add_hline(y=70, line_dash='dash', line_color='red', opacity=0.5)
    fig.add_hline(y=30, line_dash='dash', line_color='green', opacity=0.5)
    fig.update_layout(
        title=f'{symbol.upper()} RSI (14)',
        xaxis_title='Date',
        yaxis_title='RSI',
        template='plotly_white'
    )
    fig.show()


#### Get the ATR indicator of each stock in the portfolio

In [7]:
# Calculate and plot ATR (Average True Range) for each stock in the portfolio
for symbol in ticker:
	data = yf.download(symbol, period=period, interval=interval, progress=False)
	if isinstance(data.columns, pd.MultiIndex):
		data.columns = [col[0] for col in data.columns]
	data = data.reset_index()
	if 'Date' not in data.columns:
		data.rename(columns={data.columns[0]: 'Date'}, inplace=True)
	data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

	# Calculate True Range
	data['previous_close'] = data['Close'].shift(1)
	data['tr1'] = data['High'] - data['Low']
	data['tr2'] = (data['High'] - data['previous_close']).abs()
	data['tr3'] = (data['Low'] - data['previous_close']).abs()
	data['true_range'] = data[['tr1', 'tr2', 'tr3']].max(axis=1)

	# Calculate ATR
	data['atr_14'] = data['true_range'].rolling(14).mean()  # Use 14 directly or use atr_period if defined

	display(data[['Date', 'atr_14']].tail(10))

	# Plot ATR
	fig_atr = px.line(
		data,
		x=data['Date'].dt.strftime('%Y-%m-%d'),
		y='atr_14', 
		title=f'{symbol.upper()} ATR (14)'
	)
	fig_atr.update_layout(
		xaxis_title='Date',
		yaxis_title='ATR (dollars per share)',
		template='plotly_white'
	)
	display(fig_atr)

,Date,atr_14
963,2025-07-01,0.0
964,2025-07-02,0.0
965,2025-07-03,0.0
966,2025-07-07,0.0
967,2025-07-08,0.0
968,2025-07-09,0.0
969,2025-07-10,0.0
970,2025-07-11,0.0
971,2025-07-14,0.0
972,2025-07-15,0.0


,Date,atr_14
990,2025-07-01,0.003283
991,2025-07-02,0.002571
992,2025-07-03,0.002571
993,2025-07-07,0.002571
994,2025-07-08,0.002571
995,2025-07-09,0.002571
996,2025-07-10,0.002571
997,2025-07-11,0.002571
998,2025-07-14,0.002571
999,2025-07-15,0.002571


,Date,atr_14
800,2025-07-01,1.229072
801,2025-07-02,1.324786
802,2025-07-03,1.326001
803,2025-07-07,1.349572
804,2025-07-08,1.346786
805,2025-07-09,1.338929
806,2025-07-10,1.326501
807,2025-07-11,1.359644
808,2025-07-14,1.369644
809,2025-07-15,1.298215


,Date,atr_14
990,2025-07-01,0.075714
991,2025-07-02,0.079286
992,2025-07-03,0.081429
993,2025-07-07,0.077143
994,2025-07-08,0.071429
995,2025-07-09,0.069286
996,2025-07-10,0.071429
997,2025-07-11,0.077857
998,2025-07-14,0.075000
999,2025-07-15,0.067143


,Date,atr_14
990,2025-07-01,0.102143
991,2025-07-02,0.102143
992,2025-07-03,0.107143
993,2025-07-07,0.110000
994,2025-07-08,0.105715
995,2025-07-09,0.095000
996,2025-07-10,0.098572
997,2025-07-11,0.103571
998,2025-07-14,0.100714
999,2025-07-15,0.082857


...

,Date,atr_14
836,2025-07-01,0.246714
837,2025-07-02,0.235214
838,2025-07-03,0.231286
839,2025-07-07,0.224143
840,2025-07-08,0.219857
841,2025-07-09,0.203786
842,2025-07-10,0.198429
843,2025-07-11,0.277714
844,2025-07-14,0.288429
845,2025-07-15,0.299071


,Date,atr_14
990,2025-07-01,4.290930
991,2025-07-02,4.199501
992,2025-07-03,4.155573
993,2025-07-07,4.120215
994,2025-07-08,3.936786
995,2025-07-09,3.745357
996,2025-07-10,3.517500
997,2025-07-11,3.504642
998,2025-07-14,3.265356
999,2025-07-15,3.341785


,Date,atr_14
990,2025-07-01,3.163572
991,2025-07-02,3.200001
992,2025-07-03,3.250001
993,2025-07-07,3.318572
994,2025-07-08,3.442143
995,2025-07-09,3.247857
996,2025-07-10,3.480715
997,2025-07-11,3.506429
998,2025-07-14,3.474288
999,2025-07-15,3.530002


,Date,atr_14
990,2025-07-01,0.636846
991,2025-07-02,0.629113
992,2025-07-03,0.584830
993,2025-07-07,0.631925
994,2025-07-08,0.626302
995,2025-07-09,0.596780
996,2025-07-10,0.612903
997,2025-07-11,0.619843
998,2025-07-14,0.597056
999,2025-07-15,0.602579


,Date,atr_14
990,2025-07-01,0.781428
991,2025-07-02,0.789285
992,2025-07-03,0.785714
993,2025-07-07,0.806428
994,2025-07-08,0.805000
995,2025-07-09,1.172142
996,2025-07-10,1.165714
997,2025-07-11,1.217143
998,2025-07-14,1.201428
999,2025-07-15,1.210000


#### back testing
With the backtesting maybe try using blackScholes and Gbm functions along with the backtesting shit expressed in rsi-strategy.ipynb

Getting the class for backtesting

In [8]:
# Position class for trades opened/closed during the backtest
class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'

    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        if self.order_type == 'buy':
            self.profit = (self.close_price - self.open_price) * self.volume
        else:
            self.profit = (self.open_price - self.close_price) * self.volume
        self.status = 'closed'

    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }

# Strategy class for backtesting logic
class Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df

    def get_positions_df(self):
        if not self.positions:
            return pd.DataFrame()

        df = pd.DataFrame([position._asdict() for position in self.positions])

        # Replace None with 0.0 or NaN
        if 'profit' not in df.columns:
            df['profit'] = 0.0
        else:
            df['profit'] = df['profit'].fillna(0.0)
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df


    def add_position(self, position):
        self.positions.append(position)
        return True

    def close_tp_sl(self, data):
        for pos in self.positions:
            if pos.status == 'open':
                # Use 'Close' and 'Date' columns from your DataFrame
                if (pos.sl >= data['Close'] and pos.order_type == 'buy'):
                    pos.close_position(data['Date'], pos.sl)
                elif (pos.sl <= data['Close'] and pos.order_type == 'sell'):
                    pos.close_position(data['Date'], pos.sl)
                elif (pos.tp <= data['Close'] and pos.order_type == 'buy'):
                    pos.close_position(data['Date'], pos.tp)
                elif (pos.tp >= data['Close'] and pos.order_type == 'sell'):
                    pos.close_position(data['Date'], pos.tp)

    def has_open_positions(self):
        return any(pos.status == 'open' for pos in self.positions)

    def logic(self, data):
        if not self.has_open_positions():
            # Only open trades if ATR and RSI are not NaN
            if pd.notna(data['atr_14']) and pd.notna(data['rsi_14']):
                if data['rsi_14'] < 30:
                    open_datetime = data['Date']
                    open_price = data['Close']
                    order_type = 'buy'
                    volume = 10000
                    sl = open_price - 2 * data['atr_14']
                    tp = open_price + 2 * data['atr_14']
                    self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
                elif data['rsi_14'] > 70:
                    open_datetime = data['Date']
                    open_price = data['Close']
                    order_type = 'sell'
                    volume = 10000
                    sl = open_price + 2 * data['atr_14']
                    tp = open_price - 2 * data['atr_14']
                    self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))

    def run(self):
        for i, data in self.data.iterrows():
            self.close_tp_sl(data)
            self.logic(data)
        return self.get_positions_df()

In [9]:
# Loop through all tickers in your portfolio and run the backtest for each
all_results = {}

for symbol in ticker:
    print(f"\n=== Backtesting {symbol.upper()} ===")
    # Download data
    data = yf.download(symbol, period=period, interval=interval, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    if 'Date' not in data.columns:
        data.rename(columns={data.columns[0]: 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    
    # Calculate indicators
    data = rsi_calculator(data)
    # ATR calculation (reuse your code)
    data['previous_close'] = data['Close'].shift(1)
    data['tr1'] = data['High'] - data['Low']
    data['tr2'] = (data['High'] - data['previous_close']).abs()
    data['tr3'] = (data['Low'] - data['previous_close']).abs()
    data['true_range'] = data[['tr1', 'tr2', 'tr3']].max(axis=1)
    data['atr_14'] = data['true_range'].rolling(14).mean()
    
    # Run backtest
    strategy = Strategy(data, starting_balance=1000)
    results = strategy.run()
    # Ensure results DataFrame has the expected columns and order
    expected_cols = [
        'open_datetime', 'open_price', 'order_type', 'volume', 'sl', 'tp',
        'close_datetime', 'close_price', 'profit', 'status', 'pnl'
    ]
    # Always reindex to expected columns, filling missing with NaN
    results = results.reindex(columns=expected_cols, fill_value=np.nan)

    all_results[symbol] = results

    # Display results for this stock if not empty and has at least one trade
    if not results.empty and results['profit'].notna().any():
        display(results)
    else:
        print(f"No trades executed for {symbol.upper()}.")

# Now all_results contains a DataFrame for each stock


=== Backtesting CZOOF ===


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-09-28,17040.000000,buy,10000,14974.714286,19105.285714,2021-10-01,14974.714286,-2.065286e+07,closed,-2.065186e+07
1,2021-10-01,14440.000000,buy,10000,11909.171317,16970.828683,2021-10-27,16970.828683,2.530829e+07,closed,4.656430e+06
2,2021-10-27,17880.000000,sell,10000,19518.057199,16241.942801,2021-11-02,19518.057199,-1.638057e+07,closed,-1.172414e+07
3,2021-11-02,20020.000000,sell,10000,22100.914342,17939.085658,2021-11-08,17939.085658,2.080914e+07,closed,9.085001e+06
4,2021-11-15,18420.000000,sell,10000,20872.857143,15967.142857,2021-12-01,15967.142857,2.452857e+07,closed,3.361357e+07
5,2021-12-01,15600.000000,buy,10000,13873.114397,17326.885603,2021-12-09,13873.114397,-1.726886e+07,closed,1.634472e+07
6,2021-12-09,13560.000000,buy,10000,11281.685826,15838.314174,2022-01-05,11281.685826,-2.278314e+07,closed,-6.438425e+06
7,2022-01-05,10460.000000,buy,10000,9024.371512,11895.628488,2022-01-11,9024.371512,-1.435628e+07,closed,-2.079471e+07
8,2022-01-11,8980.000000,buy,10000,7479.114397,10480.885603,2022-02-23,7479.114397,-1.500886e+07,closed,-3.580357e+07
9,2022-02-23,7340.000000,buy,10000,6103.171456,8576.828544,2022-03-03,6103.171456,-1.236829e+07,closed,-4.817185e+07



=== Backtesting FCNVX ===
No trades executed for FCNVX.

=== Backtesting FDIG ===


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2022-05-10,16.868851,buy,10000,13.801210,19.936492,2022-06-13,13.801210,-30676.407994,closed,-29676.407994
1,2022-06-13,13.512847,buy,10000,11.697643,15.328051,2022-07-19,15.328051,18152.039873,closed,-11524.368121
2,2022-07-19,15.817633,sell,10000,17.514812,14.120453,2022-08-03,17.514812,-16971.791860,closed,-28496.159980
3,2022-08-03,17.885523,sell,10000,19.982461,15.788585,2022-08-12,19.982461,-20969.379154,closed,-49465.539134
4,2022-08-26,15.398133,buy,10000,13.220820,17.575447,2022-09-26,13.220820,-21773.135141,closed,-71238.674275
5,2022-09-26,13.039060,buy,10000,11.513488,14.564631,2022-10-04,14.564631,15255.715966,closed,-55982.958309
6,2022-11-07,13.518770,buy,10000,12.162832,14.874708,2022-11-09,12.162832,-13559.381653,closed,-69542.339962
7,2022-11-09,12.160576,buy,10000,10.726096,13.595056,2022-11-11,13.595056,14344.797818,closed,-55197.542144
8,2022-11-17,12.436954,buy,10000,10.912510,13.961397,2022-12-16,10.912510,-15244.432034,closed,-70441.974178
9,2022-12-16,10.620764,buy,10000,9.633704,11.607824,2023-01-09,11.607824,9870.597471,closed,-60571.376706



=== Backtesting FDKVX ===
No trades executed for FDKVX.

=== Backtesting FEDDX ===


...

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-12,207.969772,sell,10000,213.570880,202.368664,2021-08-24,202.368664,56011.078802,closed,57011.078802
1,2021-08-31,199.047455,buy,10000,193.978777,204.116132,2021-09-09,193.978777,-50686.776621,closed,6324.302181
2,2021-09-09,192.814575,buy,10000,187.255001,198.374150,2021-09-14,187.255001,-55595.745538,closed,-49271.443357
3,2021-09-14,186.719406,buy,10000,180.353883,193.084929,2021-09-30,180.353883,-63655.229938,closed,-112926.673295
4,2021-09-30,179.926605,buy,10000,172.736473,187.116737,2021-10-04,187.116737,71901.319204,closed,-41025.354091
5,2021-10-15,206.684738,sell,10000,215.253102,198.116374,2021-10-22,215.253102,-85683.640043,closed,-126708.994134
6,2021-10-22,218.094818,sell,10000,226.644833,209.544803,2021-11-23,226.644833,-85500.153175,closed,-212209.147309
7,2021-11-23,227.072327,sell,10000,233.526798,220.617856,2021-11-26,220.617856,64544.709378,closed,-147664.437930
8,2021-12-01,214.110916,buy,10000,206.178555,222.043277,2021-12-06,222.043277,79323.606691,closed,-68340.831240
9,2021-12-29,231.532104,sell,10000,237.972803,225.091406,2022-01-18,225.091406,64406.987414,closed,-3933.843825



=== Backtesting VZ ===


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-09,42.926434,buy,10000,42.137638,43.715229,2021-08-17,43.715229,7887.950690,closed,8887.950690
1,2021-08-26,42.521469,buy,10000,41.861733,43.181205,2021-10-11,41.861733,-6597.361821,closed,2290.588869
2,2021-10-11,41.119408,buy,10000,40.207049,42.031766,2021-10-20,42.031766,9123.587080,closed,11414.175949
3,2021-11-16,41.024853,buy,10000,40.057829,41.991877,2021-11-30,40.057829,-9670.239516,closed,1743.936433
4,2021-12-16,41.568596,sell,10000,43.145785,39.991407,2022-01-07,43.145785,-15771.888672,closed,-14027.952239
5,2022-01-26,40.689697,buy,10000,39.005849,42.373546,2022-01-31,42.373546,16838.487017,closed,2810.534777
6,2022-02-18,42.930744,sell,10000,44.207922,41.653566,2022-03-18,41.653566,12771.782728,closed,15582.317505
7,2022-03-18,40.514248,buy,10000,38.926032,42.102464,2022-04-06,42.102464,15882.161741,closed,31464.479246
8,2022-04-08,43.211323,sell,10000,44.345609,42.077037,2022-04-21,44.345609,-11342.857892,closed,20121.621354
9,2022-04-21,44.406033,sell,10000,45.835251,42.976814,2022-04-22,42.976814,14292.183183,closed,34413.804537



=== Backtesting WPP ===


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-16,56.639217,sell,10000,58.466287,54.812148,2021-08-19,54.812148,18270.695782,closed,19270.695782
1,2021-08-23,55.322411,sell,10000,57.130669,53.514152,2021-09-20,53.514152,18082.583139,closed,37353.278920
2,2021-09-20,53.470650,buy,10000,51.734112,55.207187,2021-09-22,55.207187,17365.376140,closed,54718.655060
3,2021-09-22,55.783291,buy,10000,53.681103,57.885478,2021-10-28,57.885478,21021.873689,closed,75740.528749
4,2021-11-01,60.678074,sell,10000,63.164382,58.191766,2021-11-30,58.191766,24863.078409,closed,100603.607158
...,...,...,...,...,...,...,...,...,...,...,...
62,2025-04-08,30.780008,buy,10000,28.429742,33.130275,2025-04-09,33.130275,23502.666704,closed,258747.677953
63,2025-04-29,36.747662,sell,10000,38.922208,34.573116,2025-05-13,38.922208,-21745.459801,closed,237002.218152
64,2025-05-13,38.929070,sell,10000,40.284043,37.574096,2025-06-09,37.574096,13549.739667,closed,250551.957819
65,2025-06-17,35.230000,buy,10000,33.765839,36.694160,2025-07-09,33.765839,-14641.607055,closed,235910.350765


#### Visualizing the backtest

In [10]:
for symbol in ticker:
    results = all_results[symbol]
    # Download data again to ensure alignment (or cache it if needed)
    data = yf.download(symbol, period=period, interval=interval, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    if 'Date' not in data.columns:
        data.rename(columns={data.columns[0]: 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

    closed_positions = results[results['status'] == 'closed']

    fig_backtest = px.line(
        data, 
        x=data['Date'].dt.strftime('%Y-%m-%d'), 
        y='Close', 
        title=f'RSI Strategy - Trades for {symbol.upper()}'
    )

    for _, position in closed_positions.iterrows():
        fig_backtest.add_shape(
            type="line",
            x0=position.open_datetime, y0=position.open_price,
            x1=position.close_datetime, y1=position.close_price,
            line=dict(
                color="green" if position.profit >= 0 else "red",
                width=3
            )
        )
    if not results.empty and results['profit'].notna().any():
        display(results)
    else:
        print(f"No trades executed for {symbol.upper()}.")
    fig_backtest.show()

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-09-28,17040.000000,buy,10000,14974.714286,19105.285714,2021-10-01,14974.714286,-2.065286e+07,closed,-2.065186e+07
1,2021-10-01,14440.000000,buy,10000,11909.171317,16970.828683,2021-10-27,16970.828683,2.530829e+07,closed,4.656430e+06
2,2021-10-27,17880.000000,sell,10000,19518.057199,16241.942801,2021-11-02,19518.057199,-1.638057e+07,closed,-1.172414e+07
3,2021-11-02,20020.000000,sell,10000,22100.914342,17939.085658,2021-11-08,17939.085658,2.080914e+07,closed,9.085001e+06
4,2021-11-15,18420.000000,sell,10000,20872.857143,15967.142857,2021-12-01,15967.142857,2.452857e+07,closed,3.361357e+07
5,2021-12-01,15600.000000,buy,10000,13873.114397,17326.885603,2021-12-09,13873.114397,-1.726886e+07,closed,1.634472e+07
6,2021-12-09,13560.000000,buy,10000,11281.685826,15838.314174,2022-01-05,11281.685826,-2.278314e+07,closed,-6.438425e+06
7,2022-01-05,10460.000000,buy,10000,9024.371512,11895.628488,2022-01-11,9024.371512,-1.435628e+07,closed,-2.079471e+07
8,2022-01-11,8980.000000,buy,10000,7479.114397,10480.885603,2022-02-23,7479.114397,-1.500886e+07,closed,-3.580357e+07
9,2022-02-23,7340.000000,buy,10000,6103.171456,8576.828544,2022-03-03,6103.171456,-1.236829e+07,closed,-4.817185e+07


No trades executed for FCNVX.


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2022-05-10,16.868851,buy,10000,13.801210,19.936492,2022-06-13,13.801210,-30676.407994,closed,-29676.407994
1,2022-06-13,13.512847,buy,10000,11.697643,15.328051,2022-07-19,15.328051,18152.039873,closed,-11524.368121
2,2022-07-19,15.817633,sell,10000,17.514812,14.120453,2022-08-03,17.514812,-16971.791860,closed,-28496.159980
3,2022-08-03,17.885523,sell,10000,19.982461,15.788585,2022-08-12,19.982461,-20969.379154,closed,-49465.539134
4,2022-08-26,15.398133,buy,10000,13.220820,17.575447,2022-09-26,13.220820,-21773.135141,closed,-71238.674275
5,2022-09-26,13.039060,buy,10000,11.513488,14.564631,2022-10-04,14.564631,15255.715966,closed,-55982.958309
6,2022-11-07,13.518770,buy,10000,12.162832,14.874708,2022-11-09,12.162832,-13559.381653,closed,-69542.339962
7,2022-11-09,12.160576,buy,10000,10.726096,13.595056,2022-11-11,13.595056,14344.797818,closed,-55197.542144
8,2022-11-17,12.436954,buy,10000,10.912510,13.961397,2022-12-16,10.912510,-15244.432034,closed,-70441.974178
9,2022-12-16,10.620764,buy,10000,9.633704,11.607824,2023-01-09,11.607824,9870.597471,closed,-60571.376706


No trades executed for FDKVX.


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-09,15.697669,buy,10000,15.482292,15.913046,2021-08-17,15.482292,-2153.772627,closed,-1153.772627
1,2021-08-17,15.308333,buy,10000,15.132009,15.484658,2021-08-19,15.132009,-1763.248444,closed,-2917.021070
2,2021-08-19,15.076390,buy,10000,14.917817,15.234964,2021-08-24,15.234964,1585.734231,closed,-1331.286839
3,2021-08-24,15.498857,buy,10000,15.298865,15.698850,2021-08-30,15.698850,1999.921799,closed,668.634960
4,2021-09-16,15.747373,buy,10000,15.558030,15.936715,2021-09-20,15.558030,-1893.422263,closed,-1224.787303
...,...,...,...,...,...,...,...,...,...,...,...
187,2025-06-12,17.740000,sell,10000,17.912857,17.567143,2025-06-13,17.567143,1728.567396,closed,-34499.119077
188,2025-06-13,17.559999,sell,10000,17.742856,17.377143,2025-06-24,17.742856,-1828.566960,closed,-36327.686037
189,2025-06-24,17.750000,sell,10000,17.961429,17.538571,2025-06-27,17.961429,-2114.287785,closed,-38441.973822
190,2025-06-27,17.980000,sell,10000,18.197142,17.762857,2025-07-02,18.197142,-2171.429225,closed,-40613.403048


...

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2022-03-18,9.850,sell,10000,9.980000,9.720000,2022-03-23,9.720000,1299.999782,closed,2299.999782
1,2022-03-23,9.710,sell,10000,9.818571,9.601429,2022-06-16,9.601429,1085.714613,closed,3385.714395
2,2022-08-05,9.670,sell,10000,9.704001,9.636000,2022-08-18,9.636000,340.005330,closed,3725.719724
3,2022-08-31,9.684,sell,10000,9.712428,9.655572,2022-09-20,9.712428,-284.280777,closed,3441.438947
4,2022-09-20,9.720,sell,10000,9.770571,9.669430,2022-10-27,9.770571,-505.707605,closed,2935.731343
5,2022-11-03,9.780,buy,10000,9.677714,9.882286,2022-11-22,9.882286,1022.860663,closed,3958.592006
6,2022-11-22,9.890,buy,10000,9.615000,10.165000,2023-04-04,10.165000,2749.998910,closed,6708.590916
7,2023-04-13,10.190,sell,10000,10.281571,10.098428,2023-06-15,10.281571,-915.712629,closed,5792.878287
8,2023-06-20,10.340,sell,10000,10.374286,10.305714,2023-06-22,10.305714,342.858178,closed,6135.736465
9,2023-06-22,10.300,sell,10000,10.337857,10.262143,2023-07-06,10.337857,-378.570557,closed,5757.165909


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-12,84.980003,buy,10000,78.723006,91.237001,2021-08-26,78.723006,-62569.972447,closed,-61569.972447
1,2021-08-26,77.739998,buy,10000,71.633570,83.846426,2021-09-10,71.633570,-61064.278739,closed,-122634.251186
2,2021-09-17,67.980003,buy,10000,62.916573,73.043434,2021-09-30,62.916573,-50634.307861,closed,-173268.559047
3,2021-09-30,60.279999,buy,10000,55.752137,64.807860,2021-10-14,64.807860,45278.614589,closed,-127989.944458
4,2021-10-21,66.989998,sell,10000,70.642283,63.337713,2021-11-01,70.642283,-36522.848947,closed,-164512.793405
5,2021-11-01,71.269997,sell,10000,75.187141,67.352852,2021-11-11,67.352852,39171.442304,closed,-125341.351100
6,2021-11-12,65.160004,buy,10000,59.955719,70.364288,2021-12-03,59.955719,-52042.846680,closed,-177384.197780
7,2021-12-03,49.990002,buy,10000,42.477001,57.503003,2022-02-17,42.477001,-75130.010332,closed,-252514.208112
8,2022-02-17,41.230000,buy,10000,37.059283,45.400716,2022-04-04,45.400716,41707.164219,closed,-210807.043893
9,2022-04-13,54.000000,sell,10000,59.903001,48.096999,2022-04-29,48.096999,59030.009678,closed,-151777.034215


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-12,207.969772,sell,10000,213.570880,202.368664,2021-08-24,202.368664,56011.078802,closed,57011.078802
1,2021-08-31,199.047455,buy,10000,193.978777,204.116132,2021-09-09,193.978777,-50686.776621,closed,6324.302181
2,2021-09-09,192.814575,buy,10000,187.255001,198.374150,2021-09-14,187.255001,-55595.745538,closed,-49271.443357
3,2021-09-14,186.719406,buy,10000,180.353883,193.084929,2021-09-30,180.353883,-63655.229938,closed,-112926.673295
4,2021-09-30,179.926605,buy,10000,172.736473,187.116737,2021-10-04,187.116737,71901.319204,closed,-41025.354091
5,2021-10-15,206.684738,sell,10000,215.253102,198.116374,2021-10-22,215.253102,-85683.640043,closed,-126708.994134
6,2021-10-22,218.094818,sell,10000,226.644833,209.544803,2021-11-23,226.644833,-85500.153175,closed,-212209.147309
7,2021-11-23,227.072327,sell,10000,233.526798,220.617856,2021-11-26,220.617856,64544.709378,closed,-147664.437930
8,2021-12-01,214.110916,buy,10000,206.178555,222.043277,2021-12-06,222.043277,79323.606691,closed,-68340.831240
9,2021-12-29,231.532104,sell,10000,237.972803,225.091406,2022-01-18,225.091406,64406.987414,closed,-3933.843825


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-09,42.926434,buy,10000,42.137638,43.715229,2021-08-17,43.715229,7887.950690,closed,8887.950690
1,2021-08-26,42.521469,buy,10000,41.861733,43.181205,2021-10-11,41.861733,-6597.361821,closed,2290.588869
2,2021-10-11,41.119408,buy,10000,40.207049,42.031766,2021-10-20,42.031766,9123.587080,closed,11414.175949
3,2021-11-16,41.024853,buy,10000,40.057829,41.991877,2021-11-30,40.057829,-9670.239516,closed,1743.936433
4,2021-12-16,41.568596,sell,10000,43.145785,39.991407,2022-01-07,43.145785,-15771.888672,closed,-14027.952239
5,2022-01-26,40.689697,buy,10000,39.005849,42.373546,2022-01-31,42.373546,16838.487017,closed,2810.534777
6,2022-02-18,42.930744,sell,10000,44.207922,41.653566,2022-03-18,41.653566,12771.782728,closed,15582.317505
7,2022-03-18,40.514248,buy,10000,38.926032,42.102464,2022-04-06,42.102464,15882.161741,closed,31464.479246
8,2022-04-08,43.211323,sell,10000,44.345609,42.077037,2022-04-21,44.345609,-11342.857892,closed,20121.621354
9,2022-04-21,44.406033,sell,10000,45.835251,42.976814,2022-04-22,42.976814,14292.183183,closed,34413.804537


,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-08-16,56.639217,sell,10000,58.466287,54.812148,2021-08-19,54.812148,18270.695782,closed,19270.695782
1,2021-08-23,55.322411,sell,10000,57.130669,53.514152,2021-09-20,53.514152,18082.583139,closed,37353.278920
2,2021-09-20,53.470650,buy,10000,51.734112,55.207187,2021-09-22,55.207187,17365.376140,closed,54718.655060
3,2021-09-22,55.783291,buy,10000,53.681103,57.885478,2021-10-28,57.885478,21021.873689,closed,75740.528749
4,2021-11-01,60.678074,sell,10000,63.164382,58.191766,2021-11-30,58.191766,24863.078409,closed,100603.607158
...,...,...,...,...,...,...,...,...,...,...,...
62,2025-04-08,30.780008,buy,10000,28.429742,33.130275,2025-04-09,33.130275,23502.666704,closed,258747.677953
63,2025-04-29,36.747662,sell,10000,38.922208,34.573116,2025-05-13,38.922208,-21745.459801,closed,237002.218152
64,2025-05-13,38.929070,sell,10000,40.284043,37.574096,2025-06-09,37.574096,13549.739667,closed,250551.957819
65,2025-06-17,35.230000,buy,10000,33.765839,36.694160,2025-07-09,33.765839,-14641.607055,closed,235910.350765


#### Plotting the profit & loss 

In [11]:
for symbol in ticker:
    results = all_results[symbol]
    if not results.empty and results['pnl'].notna().any():
        closed_results = results[results['status'] == 'closed']
        if not closed_results.empty:
            # Format close_datetime as string for cleaner x-axis
            closed_results = closed_results.copy()
            closed_results['formatted_date'] = closed_results['close_datetime'].dt.strftime('%Y-%m-%d')

            fig_pnl = px.line(
                closed_results,
                x='formatted_date',
                y='pnl',
                title=f'PnL Curve for {symbol.upper()}'
            )
            fig_pnl.update_layout(
                xaxis_title='Date',
                yaxis_title='PnL'
            )
            fig_pnl.show()
        else:
            print(f"No closed trades for {symbol.upper()}.")
    else:
        print(f"No PnL data for {symbol.upper()}.")


No PnL data for FCNVX.


No PnL data for FDKVX.


No PnL data for FSENX.


#### Bayesian RSI indiator

In [12]:
def bayesian_rsi_signal(rsi_value):
    # Prior: equal probability for each action
    probs = {'buy': 1/3, 'hold': 1/3, 'sell': 1/3}
    # Likelihoods based on RSI value
    if rsi_value < 30:
        probs['buy'] *= 0.8
        probs['hold'] *= 0.15
        probs['sell'] *= 0.05
    elif rsi_value > 70:
        probs['buy'] *= 0.05
        probs['hold'] *= 0.15
        probs['sell'] *= 0.8
    else:
        probs['buy'] *= 0.15
        probs['hold'] *= 0.7
        probs['sell'] *= 0.15
    # Normalize
    total = sum(probs.values())
    for k in probs:
        probs[k] /= total
    # Choose the action with the highest probability
    signal = max(probs, key=probs.get)
    return signal, probs

# Example usage for the latest RSI value of each stock:
for symbol in ticker:
    data = yf.download(symbol, period=period, interval=interval, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    if 'Date' not in data.columns:
        data.rename(columns={data.columns[0]: 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    data = rsi_calculator(data)
    latest_rsi = data['rsi_14'].iloc[-1]
    signal, probs = bayesian_rsi_signal(latest_rsi)
    print(f"{symbol.upper()} | RSI: {latest_rsi:.2f} | Signal: {signal} | Probabilities: {probs}")

CZOOF | RSI: 99.98 | Signal: sell | Probabilities: {'buy': 0.05, 'hold': 0.15, 'sell': 0.8}
FCNVX | RSI: nan | Signal: hold | Probabilities: {'buy': 0.15, 'hold': 0.7, 'sell': 0.15}
FDIG | RSI: 50.77 | Signal: hold | Probabilities: {'buy': 0.15, 'hold': 0.7, 'sell': 0.15}
FDKVX | RSI: nan | Signal: hold | Probabilities: {'buy': 0.15, 'hold': 0.7, 'sell': 0.15}
FEDDX | RSI: 22.02 | Signal: buy | Probabilities: {'buy': 0.8, 'hold': 0.15, 'sell': 0.05}
FFRHX | RSI: 10.32 | Signal: buy | Probabilities: {'buy': 0.8, 'hold': 0.15, 'sell': 0.05}
FIFNX | RSI: 4.68 | Signal: buy | Probabilities: {'buy': 0.8, 'hold': 0.15, 'sell': 0.05}
FITLX | RSI: 0.00 | Signal: buy | Probabilities: {'buy': 0.8, 'hold': 0.15, 'sell': 0.05}
FLOWX | RSI: 100.00 | Signal: sell | Probabilities: {'buy': 0.05, 'hold': 0.15, 'sell': 0.8}
FSENX | RSI: nan | Signal: hold | Probabilities: {'buy': 0.15, 'hold': 0.7, 'sell': 0.15}
LION | RSI: 71.13 | Signal: sell | Probabilities: {'buy': 0.05, 'hold': 0.15, 'sell': 0.8}
O

## Back testing

https://youtu.be/yiy2WneA3Ng?si=nE4ebfqlRMwnBsXl

Look into the strategies I would need to do the long only strategy using moving average cross overs and how far I am to actuallu already beung there  visualize. try to display sma 10 and sma 200 on the grtrading graphs and try to get it to candlestick charts

## Bollinger bands

## MACD